In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import regex as re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from textblob import TextBlob

In [2]:
import vaex

In [18]:
#reviews = pd.read_csv('./yelp_reviews.csv')

In [74]:
#reviews.head(10)

,business_id,stars,useful,funny,cool,text
0,--1UhMGODdWsrMastO9DZw,5,0,0,0,I just love this little Mexican cafeteria-styl...
1,--1UhMGODdWsrMastO9DZw,3,0,0,0,"We got guacamole, both the vegetarian tacos (R..."
2,--1UhMGODdWsrMastO9DZw,4,0,0,1,Downtown west which honestly feels like Downto...
3,--1UhMGODdWsrMastO9DZw,5,0,0,0,Glad these guys moved in on the block! It's qu...
4,--1UhMGODdWsrMastO9DZw,5,2,1,2,Fantastic spot for lunch with great value for ...
5,--1UhMGODdWsrMastO9DZw,5,0,0,1,In my last review I mentioned getting charged ...
6,--1UhMGODdWsrMastO9DZw,5,0,0,1,A little hole in the wall for some really deli...
7,--1UhMGODdWsrMastO9DZw,5,0,0,1,If you want Mexican for a reasonable price in ...
8,--1UhMGODdWsrMastO9DZw,5,1,0,0,Probably my favorite Mexican place for tacos i...
9,--1UhMGODdWsrMastO9DZw,4,2,0,0,Pros:\n- Super Clean\n- Authentic recipes for ...


In [3]:
df = vaex.from_csv('./yelp_reviews.csv', convert=True, copy_index=False)

In [4]:
df.head()

#,business_id,stars,useful,funny,cool,text
<i style='opacity: 0.6'>0</i>,--1UhMGODdWsrMastO9DZw,5,0,0,0,'I just love this little Mexican cafeteria-style...
<i style='opacity: 0.6'>1</i>,--1UhMGODdWsrMastO9DZw,3,0,0,0,"'We got guacamole, both the vegetarian tacos (Ra..."
<i style='opacity: 0.6'>2</i>,--1UhMGODdWsrMastO9DZw,4,0,0,1,'Downtown west which honestly feels like Downtow...
<i style='opacity: 0.6'>3</i>,--1UhMGODdWsrMastO9DZw,5,0,0,0,"""Glad these guys moved in on the block! It's qui..."
<i style='opacity: 0.6'>4</i>,--1UhMGODdWsrMastO9DZw,5,2,1,2,'Fantastic spot for lunch with great value for y...
<i style='opacity: 0.6'>5</i>,--1UhMGODdWsrMastO9DZw,5,0,0,1,'In my last review I mentioned getting charged f...
<i style='opacity: 0.6'>6</i>,--1UhMGODdWsrMastO9DZw,5,0,0,1,'A little hole in the wall for some really delic...
<i style='opacity: 0.6'>7</i>,--1UhMGODdWsrMastO9DZw,5,0,0,1,'If you want Mexican for a reasonable price in t...
<i style='opacity: 0.6'>8</i>,--1UhMGODdWsrMastO9DZw,5,1,0,0,'Probably my favorite Mexican place for tacos in...
<i style='opacity: 0.6'>9</i>,--1UhMGODdWsrMastO9DZw,4,2,0,0,'Pros:\n- Super Clean\n- Authentic recipes for the...


In [5]:
df.dtypes

business_id    <class 'str'>
stars                  int64
useful                 int64
funny                  int64
cool                   int64
text           <class 'str'>
dtype: object

In [6]:
#reviews['text'][0:10].apply(lambda Text:pd.Series(TextBlob(Text).sentiment.polarity))

In [6]:
df.drop(columns=['stars', 'useful', 'funny', 'cool'], inplace=True)

#,business_id,text
<i style='opacity: 0.6'>0</i>,--1UhMGODdWsrMastO9DZw,'I just love this little Mexican cafeteria-style...
<i style='opacity: 0.6'>1</i>,--1UhMGODdWsrMastO9DZw,"'We got guacamole, both the vegetarian tacos (Ra..."
<i style='opacity: 0.6'>2</i>,--1UhMGODdWsrMastO9DZw,'Downtown west which honestly feels like Downtow...
<i style='opacity: 0.6'>3</i>,--1UhMGODdWsrMastO9DZw,"""Glad these guys moved in on the block! It's qui..."
<i style='opacity: 0.6'>4</i>,--1UhMGODdWsrMastO9DZw,'Fantastic spot for lunch with great value for y...
...,...,...
"<i style='opacity: 0.6'>5,056,222</i>",zzzaIBwimxVej4tY6qFOUQ,"'After reading all these good reviews, I was pre..."
"<i style='opacity: 0.6'>5,056,223</i>",zzzaIBwimxVej4tY6qFOUQ,'This place is very good. At first i thought it ...
"<i style='opacity: 0.6'>5,056,224</i>",zzzaIBwimxVej4tY6qFOUQ,"""Guthrie's is a great choice for people who are ..."
"<i style='opacity: 0.6'>5,056,225</i>",zzzaIBwimxVej4tY6qFOUQ,"'This location used to be a KFC, I went there on..."


In [17]:
df = df.groupby('business_id')['text'].apply(','.join)

TypeError: 'GroupBy' object is not subscriptable

In [7]:
%%time
df['polarity'] = df['text'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment.polarity).values[0])

Wall time: 0 ns


In [8]:
df['polarity']

Expression = polarity
Length: 5,056,227 dtype: float64 (column)
-----------------------------------------
      0   0.125105
      1     0.1455
      2   0.187738
      3   0.397024
      4   0.258785
       ...        
5056222  0.0336332
5056223   0.291806
5056224       0.17
5056225   0.140661
5056226      0.025

In [10]:
%%time
#Using Vaex
df['subjectivity'] = df['text'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment.subjectivity).values[0])

Wall time: 998 µs


In [11]:
df

#,business_id,stars,useful,funny,cool,text,polarity,subjectivity
<i style='opacity: 0.6'>0</i>,--1UhMGODdWsrMastO9DZw,5,0,0,0,'I just love this little Mexican cafeteria-style...,0.1251046176046176,0.4671212121212121
<i style='opacity: 0.6'>1</i>,--1UhMGODdWsrMastO9DZw,3,0,0,0,"'We got guacamole, both the vegetarian tacos (Ra...",0.1455,0.49866666666666665
<i style='opacity: 0.6'>2</i>,--1UhMGODdWsrMastO9DZw,4,0,0,1,'Downtown west which honestly feels like Downtow...,0.18773809523809523,0.47130952380952384
<i style='opacity: 0.6'>3</i>,--1UhMGODdWsrMastO9DZw,5,0,0,0,"""Glad these guys moved in on the block! It's qui...",0.39702380952380956,0.5285714285714286
<i style='opacity: 0.6'>4</i>,--1UhMGODdWsrMastO9DZw,5,2,1,2,'Fantastic spot for lunch with great value for y...,0.25878489326765186,0.5093596059113301
...,...,...,...,...,...,...,...,...
"<i style='opacity: 0.6'>5,056,222</i>",zzzaIBwimxVej4tY6qFOUQ,2,2,0,0,"'After reading all these good reviews, I was pre...",0.03363315696649031,0.5797795414462081
"<i style='opacity: 0.6'>5,056,223</i>",zzzaIBwimxVej4tY6qFOUQ,4,0,0,0,'This place is very good. At first i thought it ...,0.29180555555555554,0.5156944444444445
"<i style='opacity: 0.6'>5,056,224</i>",zzzaIBwimxVej4tY6qFOUQ,5,2,0,0,"""Guthrie's is a great choice for people who are ...",0.17,0.7499999999999999
"<i style='opacity: 0.6'>5,056,225</i>",zzzaIBwimxVej4tY6qFOUQ,3,1,0,0,"'This location used to be a KFC, I went there on...",0.14066142191142192,0.6046620046620046


In [12]:
df['polarity']

Expression = polarity
Length: 5,056,227 dtype: float64 (column)
-----------------------------------------
      0   0.125105
      1     0.1455
      2   0.187738
      3   0.397024
      4   0.258785
       ...        
5056222  0.0336332
5056223   0.291806
5056224       0.17
5056225   0.140661
5056226      0.025

In [11]:
pol_vals = df['polarity'][0:10].values[0]

In [17]:
df['polarity'][0:10].values[0][0]

0.1251046176046176

In [ ]:
df['polarity_values'] = df['polarity'].values

In [13]:
#just_in_case = df.groupby(df.business_id, agg=[vaex.agg.mean(df.polarity.values[0][0])])#, vaex.agg.mean('y')])

In [23]:
materia.dtypes

business_id     <class 'str'>
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text            <class 'str'>
subjectivity          float64
polarity              float64
dtype: object

In [24]:
df.dtypes

business_id     <class 'str'>
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text            <class 'str'>
polarity              float64
subjectivity          float64
dtype: object

In [16]:
df.groupby(df.business_id, agg=vaex.agg.mean('polarity'))

ERROR:MainThread:vaex.execution:error in task, flush task queue
Traceback (most recent call last):
  File "C:\Users\Luke\anaconda3\lib\site-packages\vaex\scopes.py", line 97, in evaluate
    result = self[expression]
  File "C:\Users\Luke\anaconda3\lib\site-packages\vaex\scopes.py", line 144, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: 'Unknown variables or column: "astype(polarity, \'float64\')"'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Luke\anaconda3\lib\site-packages\vaex\scopes.py", line 97, in evaluate
    result = self[expression]
  File "C:\Users\Luke\anaconda3\lib\site-packages\vaex\scopes.py", line 144, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'lambda_function(text)'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'NoneType'>

In [27]:
reviews['polarity'] = df['polarity']

In [34]:
df['polarity']

TypeError: list indices must be integers or slices, not str

In [31]:
reviews

,business_id,stars,useful,funny,cool,text,polarity
0,--1UhMGODdWsrMastO9DZw,5,0,0,0,I just love this little Mexican cafeteria-styl...,polarity
1,--1UhMGODdWsrMastO9DZw,3,0,0,0,"We got guacamole, both the vegetarian tacos (R...",polarity
2,--1UhMGODdWsrMastO9DZw,4,0,0,1,Downtown west which honestly feels like Downto...,polarity
3,--1UhMGODdWsrMastO9DZw,5,0,0,0,Glad these guys moved in on the block! It's qu...,polarity
4,--1UhMGODdWsrMastO9DZw,5,2,1,2,Fantastic spot for lunch with great value for ...,polarity
...,...,...,...,...,...,...,...
5056222,zzzaIBwimxVej4tY6qFOUQ,2,2,0,0,"After reading all these good reviews, I was pr...",polarity
5056223,zzzaIBwimxVej4tY6qFOUQ,4,0,0,0,This place is very good. At first i thought it...,polarity
5056224,zzzaIBwimxVej4tY6qFOUQ,5,2,0,0,Guthrie's is a great choice for people who are...,polarity
5056225,zzzaIBwimxVej4tY6qFOUQ,3,1,0,0,"This location used to be a KFC, I went there o...",polarity


In [33]:
polarity

Expression = polarity
Length: 5,056,227 dtype: float64 (column)
-----------------------------------------
      0  [0.12510462]
      1      [0.1455]
      2   [0.1877381]
      3  [0.39702381]
      4  [0.25878489]
         ...         
5056222  [0.03363316]
5056223  [0.29180556]
5056224        [0.17]
5056225  [0.14066142]
5056226       [0.025]